## Set up matching DF's

In [227]:
import pandas as pd
# Set pandas to display all columns
pd.set_option('display.max_columns', None)
import os
import re

In [228]:
directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/Toto"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('totoFootball') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_toto = max(files, key=os.path.getmtime)

directory = "/Users/ddeboe01/Downloads/ArbitrageBets/Data/scrapers/unibet"
file_extension = ".csv"
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(file_extension.split('.')[-1]) and (file.startswith('toto') or file.startswith('unibet'))]
if not files:
    raise FileNotFoundError(f"No files found in {directory} with extension {file_extension}")
latest_file_unibet = max(files, key=os.path.getmtime)

In [229]:
toto_raw = pd.read_csv(f"{latest_file_toto}", index_col=0).drop_duplicates()
kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
# Apply the condition and update outcome_label
toto_raw_football = toto_raw[toto_raw['sport'] == 'Voetbal']
kambi_raw_football = kambi_raw[kambi_raw['sport'] == 'FOOTBALL']
# Adjust odds and line
kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
kambi_raw_football['odds'] = kambi_raw_football['odds'] / 1000

# Adjust toto outcome
toto_raw_football['Outcome SubType'] = toto_raw_football['Outcome SubType'].replace({'H': '1', 'A': '2'})

# List of women's competitions
women_competitions_toto = [
    'Portugal Campeonato Nacional, Vrouwen', 'Mexico League MX Vrouwen', 'Australië W-League', 'Italië Coppa Italia Vrouwen',
    'Scotland Women\'s Premier League', 'Nederland Eredivisie Vrouwen', 'England FA Cup Women', 'Engeland FA Super League Vrouwen',
    'Spain Primera División Vrouwen'
]

# Create 'sex' column based on the competition
toto_raw_football['sex'] = toto_raw_football['competition'].apply(
    lambda x: 'W' if x in women_competitions_toto else 'M'
)

# List of women's football competitions
women_competitions_kambi = [
    'A-League (D)', 'Premier League Dames', 'Campeonato Nacional Feminino', 'Liga MX Femenil (D)', 
    'Frauen-Bundesliga', 'Super League (D)', 'Primera División (D)', 'Coppa Italia (D)', 'Liga MX Femenil'

]

# Create 'sex' column
kambi_raw_football['sex'] = kambi_raw_football['group_name'].apply(
    lambda x: 'W' if x in women_competitions_kambi else 'M'
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/3251815356.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  kambi_raw = pd.read_csv(f"{latest_file_unibet}", index_col=0).drop_duplicates()
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/3251815356.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/3251815356.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [230]:
kambi_raw[kambi_raw['event_name'].str.contains('Club Brugge')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time
53707,2489553704,1001159668,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1022540293,3626720229,1X,1X,1120.0,NaN,NaN,OT_ONE_OR_CROSS,2025-02-05T20:27:01Z,1/9,-835.0,OPEN,ENABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie,2025-02-05T19:50:00Z
53708,2489553704,1001159668,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1022540293,3626720233,12,12,3400.0,NaN,NaN,OT_ONE_OR_TWO,2025-02-05T20:27:01Z,12/5,240.0,OPEN,ENABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie,2025-02-05T19:50:00Z
53709,2489553704,1001159668,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1022540293,3626720241,X2,X2,1120.0,NaN,NaN,OT_CROSS_OR_TWO,2025-02-05T20:27:01Z,1/9,-835.0,OPEN,ENABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie,2025-02-05T19:50:00Z
53710,2489553709,1001159922,Dubbele Kans,Double Chance,GOALS,FULL_TIME,12,Dubbele kans,Double Chance,1022540293,3626720237,1X,1X,1480.0,NaN,NaN,OT_ONE_OR_CROSS,2025-02-05T20:22:03Z,12/25,-210.0,OPEN,ENABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie,2025-02-05T19:50:00Z
53711,2489553709,1001159922,Dubbele Kans,Double Chance,GOALS,FULL_TIME,12,Dubbele kans,Double Chance,1022540293,3626720245,12,12,1300.0,NaN,NaN,OT_ONE_OR_TWO,2025-02-05T20:22:03Z,3/10,-335.0,OPEN,ENABLED,NaN,NaN,KRC Genk vs Club Brugge,FOOTBALL,Beker van Belgie,2025-02-05T19:50:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55573,2490440318,1005153918,Eerste Doelpuntenmaker,First Goal Scorer,GOALS,FULL_TIME,125,Player Occurrence Number,Player Occurrence Number,1020999860,3629958269,"Kurucay, Hasan","Kurucay, Hasan",41000.0,NaN,"Kurucay, Hasan",OT_PLAYER_PARTICIPANT,2025-02-05T12:59:02Z,40/1,4000.0,OPEN,ENABLED,NaN,NaN,Club Brugge vs OH Leuven,FOOTBALL,Jupiler Pro League,2025-02-08T15:00:00Z
55574,2490440318,1005153918,Eerste Doelpuntenmaker,First Goal Scorer,GOALS,FULL_TIME,125,Player Occurrence Number,Player Occurrence Number,1020999860,3629958271,"Verstraete, Birger","Verstraete, Birger",26000.0,NaN,"Verstraete, Birger",OT_PLAYER_PARTICIPANT,2025-02-05T12:59:02Z,25/1,2500.0,OPEN,ENABLED,NaN,NaN,Club Brugge vs OH Leuven,FOOTBALL,Jupiler Pro League,2025-02-08T15:00:00Z
55575,2490440318,1005153918,Eerste Doelpuntenmaker,First Goal Scorer,GOALS,FULL_TIME,125,Player Occurrence Number,Player Occurrence Number,1020999860,3629958273,"Mitrovic, Stefan (2002)","Mitrovic, Stefan (2002)",11000.0,NaN,"Mitrovic, Stefan (2002)",OT_PLAYER_PARTICIPANT,2025-02-05T12:59:02Z,10/1,1000.0,OPEN,ENABLED,NaN,NaN,Club Brugge vs OH Leuven,FOOTBALL,Jupiler Pro League,2025-02-08T15:00:00Z
55576,2490440318,1005153918,Eerste Doelpuntenmaker,First Goal Scorer,GOALS,FULL_TIME,125,Player Occurrence Number,Player Occurrence Number,1020999860,3629958276,"Ricca, Federico","Ricca, Federico",46000.0,NaN,"Ricca, Federico",OT_PLAYER_PARTICIPANT,2025-02-03T02:48:49Z,45/1,4500.0,OPEN,ENABLED,NaN,NaN,Club Brugge vs OH Leuven,FOOTBALL,Jupiler Pro League,2025-02-08T15:00:00Z


Below is a translation of the abbreviations for the column outcome types into their likely meanings in the context of betting odds:

- `--`: Undefined or No Outcome (e.g., no odds available).
- `AG`: Anytime Goal Scorer (player to score at any time in the match).
- `CS`: Correct Score (predict the exact final score of the event).
- `DC`: Double Chance (cover two outcomes, e.g., Home Win or Draw).
- `DN`: Draw No Bet (stake refunded if the match ends in a draw).
- `FS`: First Scorer (player to score the first goal/point in the event).
- `H1`: First Half Result (outcome at the end of the first half).
- `H2`: Second Half Result (outcome in the second half only).
- `HF`: Half-Time/Full-Time (predict the result at both half-time and full-time).
- `HH`: Head-to-Head (comparison between two participants, e.g., players or teams).
- `HL`: Handicap Line (spread betting; adjust the line to level the playing field).
- `LS`: Last Scorer (player to score the last goal/point in the event).
- `MH`: Match Handicap (spread betting for the entire match).
- `MR`: Match Result (predict the overall winner or draw for the match).
- `OE`: Odd/Even (predict whether the total points/goals scored will be odd or even).
- `WH`: Winning Half (which half will have the higher score).
- `WM`: Winning Margin (predict the margin by which a team/player will win).

In [231]:
# # filter down to suitable betting opps
# kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

# kambi_filtered_football = kambi_filtered_football
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football

# Remove duplicates in toto_filtered_football based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_football based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

# Replace 'Sport Club' with 'SC' in the 'event_name' column
kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/1898072072.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)


## Winnaar: Draw no Bet

In [232]:
from rapidfuzz import process, fuzz
import pandas as pd

# Filter kambi_filtered_football based on criterion_label
filtered_kambi_winnaar = kambi_filtered_football[
    kambi_filtered_football['criterion_label'].str.contains('Draw No Bet')
]

# Filter kambi_filtered_football based on criterion_label
filtered_toto_winnaar = toto_filtered_football[
    toto_filtered_football['Market Name'].str.contains('Draw No Bet')
]

# Preprocess strings: Remove extra whitespace and lowercase
def preprocess(text):
    return ' '.join(text.lower().split())

In [233]:
import unicodedata
from fuzzywuzzy import process, fuzz

# Replace '-' with a space and remove accents in the 'names' column
filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Create 'Team1' and 'Team2'
filtered_kambi_winnaar[['Team1', 'Team2']] = filtered_kambi_winnaar['event_name'].str.split(' vs ', expand=True)
filtered_toto_winnaar[['Team1', 'Team2']] = filtered_toto_winnaar['Event Name'].str.split(' vs ', expand=True)

kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# # Function to find the best match based on Team1 and Team2 fuzzy matching
# def find_best_match(event_name):
#     # Extract Team1 and Team2 from the event_name
#     parts = event_name.split(' vs ')
#     if len(parts) == 2:
#         team1 = parts[0].strip()
#         team2 = parts[1].strip()

#         # Perform fuzzy matching for Team1 and Team2
#         def match_teams(team, kambi_team):
#             return fuzz.token_set_ratio(team, kambi_team) >= 80

#         # Check each event_name in kambi_events
#         for kambi_event in kambi_events:
#             kambi_parts = kambi_event.split(' vs ')
#             if len(kambi_parts) == 2:
#                 kambi_team1 = kambi_parts[0].strip()
#                 kambi_team2 = kambi_parts[1].strip()

#                 # Check if both teams match
#                 if match_teams(team1, kambi_team1) and match_teams(team2, kambi_team2):
#                     return kambi_event  # Return the matched event_name

#     return None  # Return None if no match found

from fuzzywuzzy import fuzz

# Function to find the best match based on Team1 and Team2 fuzzy matching
def find_best_match(event_name):
    # Extract Team1 and Team2 from the event_name
    parts = event_name.split(' vs ')
    if len(parts) == 2:
        team1 = parts[0].strip().replace('Athletic', '').replace('Atletico', '')
        team2 = parts[1].strip().replace('Athletic', '').replace('Atletico', '')

        # Perform fuzzy matching for Team1 and Team2 with all kambi events
        def match_teams(team, kambi_team):
            return fuzz.token_set_ratio(team, kambi_team)

        # Perform fuzzy matching for both Team1 and Team2
        kambi_matches = filtered_kambi_winnaar[
            (filtered_kambi_winnaar['Team1'].apply(lambda x: match_teams(team1, x) >= 65)) &
            (filtered_kambi_winnaar['Team2'].apply(lambda x: match_teams(team2, x) >= 65))
        ]
        
        # If there is a match, return the matched event_name and fuzzy match score
        if not kambi_matches.empty:
            # Get the best match score
            best_score_team1 = kambi_matches['Team1'].apply(lambda x: match_teams(team1, x)).max()
            best_score_team2 = kambi_matches['Team2'].apply(lambda x: match_teams(team2, x)).max()

            # Calculate the average score (or you can choose to return another metric)
            average_score = (best_score_team1 + best_score_team2) / 2

            # Get the matched event name
            matched_event_name = kambi_matches['event_name'].iloc[0]

            return matched_event_name, average_score  # Return the matched event and score

    return None, None  # Return None if no match found

# Apply matching function to toto_filtered_football
filtered_toto_winnaar[['matched_event', 'fuzzy_score']] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: pd.Series(find_best_match(x))
)

# # Apply matching function to filtered_toto_winnaar
# filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

# Check the results
filtered_toto_winnaar[['Event Name', 'matched_event']].head()

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/1566042253.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/1566042253.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/1566042253.py:20: SettingW

,Event Name,matched_event
event_id,,
6559526,Benfica vs Moreirense,Benfica vs Moreirense
6559526,Benfica vs Moreirense,Benfica vs Moreirense
6559526,Benfica vs Moreirense,Benfica vs Moreirense
6559526,Benfica vs Moreirense,Benfica vs Moreirense
6680353,Tottenham Hotspur vs Manchester United,Tottenham vs Manchester United


In [234]:
filtered_toto_winnaar[filtered_toto_winnaar['Event Name'].str.contains('Brugge')]

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex,Team1,Team2,matched_event,fuzzy_score
event_id,,,,,,,,,,,,,,,,,,,
6416615,Club Brugge vs Oud Heverlee Leuven,Draw No Bet,Oud-Heverlee Leuven,5.75,19,4,DN,2,Voetbal,België Jupiler Pro League,Club Brugge vs Oud-Heverlee Leuven,Club Brugge,Oud-Heverlee Leuven,2025-02-08T15:00:00Z,M,Club Brugge,Oud Heverlee Leuven,Club Brugge vs OH Leuven,90.0
6416615,Club Brugge vs Oud Heverlee Leuven,Draw No Bet,Club Brugge,1.11,11,100,DN,1,Voetbal,België Jupiler Pro League,Club Brugge vs Oud-Heverlee Leuven,Club Brugge,Oud-Heverlee Leuven,2025-02-08T15:00:00Z,M,Club Brugge,Oud Heverlee Leuven,Club Brugge vs OH Leuven,90.0
6416615,Club Brugge vs Oud Heverlee Leuven,1e Helft: Draw No Bet,Club Brugge,1.16,4,25,DN,1,Voetbal,België Jupiler Pro League,Club Brugge vs Oud-Heverlee Leuven,Club Brugge,Oud-Heverlee Leuven,2025-02-08T15:00:00Z,M,Club Brugge,Oud Heverlee Leuven,Club Brugge vs OH Leuven,90.0
6416615,Club Brugge vs Oud Heverlee Leuven,1e Helft: Draw No Bet,Oud-Heverlee Leuven,4.50,7,2,DN,2,Voetbal,België Jupiler Pro League,Club Brugge vs Oud-Heverlee Leuven,Club Brugge,Oud-Heverlee Leuven,2025-02-08T15:00:00Z,M,Club Brugge,Oud Heverlee Leuven,Club Brugge vs OH Leuven,90.0
6416615,Club Brugge vs Oud Heverlee Leuven,2e Helft: Draw No Bet,Oud-Heverlee Leuven,4.25,13,4,DN,2,Voetbal,België Jupiler Pro League,Club Brugge vs Oud-Heverlee Leuven,Club Brugge,Oud-Heverlee Leuven,2025-02-08T15:00:00Z,M,Club Brugge,Oud Heverlee Leuven,Club Brugge vs OH Leuven,90.0
6416615,Club Brugge vs Oud Heverlee Leuven,2e Helft: Draw No Bet,Club Brugge,1.18,9,50,DN,1,Voetbal,België Jupiler Pro League,Club Brugge vs Oud-Heverlee Leuven,Club Brugge,Oud-Heverlee Leuven,2025-02-08T15:00:00Z,M,Club Brugge,Oud Heverlee Leuven,Club Brugge vs OH Leuven,90.0
6416613,KRC Genk vs Cercle Brugge,Draw No Bet,KRC Genk,1.25,1,4,DN,1,Voetbal,België Jupiler Pro League,KRC Genk vs Cercle Brugge,KRC Genk,Cercle Brugge,2025-02-08T19:45:00Z,M,KRC Genk,Cercle Brugge,KRC Genk vs Club Brugge,100.0
6416613,KRC Genk vs Cercle Brugge,Draw No Bet,Cercle Brugge,3.55,51,20,DN,2,Voetbal,België Jupiler Pro League,KRC Genk vs Cercle Brugge,KRC Genk,Cercle Brugge,2025-02-08T19:45:00Z,M,KRC Genk,Cercle Brugge,KRC Genk vs Club Brugge,100.0
6416613,KRC Genk vs Cercle Brugge,1e Helft: Draw No Bet,KRC Genk,1.33,33,100,DN,1,Voetbal,België Jupiler Pro League,KRC Genk vs Cercle Brugge,KRC Genk,Cercle Brugge,2025-02-08T19:45:00Z,M,KRC Genk,Cercle Brugge,KRC Genk vs Club Brugge,100.0


In [235]:
filtered_kambi_winnaar

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex,Team1,Team2
43,2489310292,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021152032,3625758815,1,1,2.40,NaN,Como,1,2025-02-05T11:43:44Z,7/5,140.0,OPEN,DISABLED,NaN,NaN,Como vs Juventus,FOOTBALL,Serie A,2025-02-07T19:45:00Z,M,Como,Juventus
44,2489310292,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021152032,3625758817,2,2,1.50,NaN,Juventus,2,2025-02-05T09:03:24Z,1/2,-200.0,OPEN,DISABLED,NaN,NaN,Como vs Juventus,FOOTBALL,Serie A,2025-02-07T19:45:00Z,M,Como,Juventus
242,2489310640,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021152032,3625759547,1,1,2.70,NaN,Como,1,2025-02-05T09:03:22Z,17/10,170.0,OPEN,DISABLED,NaN,NaN,Como vs Juventus,FOOTBALL,Serie A,2025-02-07T19:45:00Z,M,Como,Juventus
243,2489310640,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021152032,3625759549,2,2,1.45,NaN,Juventus,2,2025-02-05T11:43:46Z,4/9,-225.0,OPEN,DISABLED,NaN,NaN,Como vs Juventus,FOOTBALL,Serie A,2025-02-07T19:45:00Z,M,Como,Juventus
350,2491140731,1001421321,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2,Wedstrijd,Match,1021152032,3632837089,1,1,2.33,NaN,Como,1,2025-02-05T11:43:46Z,13/10,133.0,OPEN,DISABLED,NaN,NaN,Como vs Juventus,FOOTBALL,Serie A,2025-02-07T19:45:00Z,M,Como,Juventus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139413,2491466248,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021192930,3633912570,2,2,2.75,NaN,AFC Fylde,2,2025-02-05T07:43:56Z,7/4,175.0,OPEN,ENABLED,NaN,NaN,Aldershot vs AFC Fylde,FOOTBALL,National League,2025-02-08T15:00:00Z,M,Aldershot,AFC Fylde
139457,2491456505,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021192947,3633902539,1,1,2.06,NaN,Tamworth,1,2025-02-05T07:44:31Z,21/20,106.0,OPEN,DISABLED,NaN,NaN,Tamworth vs Halifax Town,FOOTBALL,National League,2025-02-08T17:30:00Z,M,Tamworth,Halifax Town
139458,2491456505,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021192947,3633902545,2,2,1.68,NaN,Halifax Town,2,2025-02-05T07:44:31Z,4/6,-148.0,OPEN,DISABLED,NaN,NaN,Tamworth vs Halifax Town,FOOTBALL,National League,2025-02-08T17:30:00Z,M,Tamworth,Halifax Town
139461,2491456507,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021192947,3633902568,1,1,2.00,NaN,Tamworth,1,2025-02-05T07:44:31Z,Evens,100.0,OPEN,ENABLED,NaN,NaN,Tamworth vs Halifax Town,FOOTBALL,National League,2025-02-08T17:30:00Z,M,Tamworth,Halifax Town


In [236]:
# Get unique records from 'Event Name' and 'matched_event'
matched_events = filtered_toto_winnaar[['Event Name', 'matched_event', 'fuzzy_score']].drop_duplicates()

# Display the result
matched_events[~matched_events['matched_event'].isna()].sort_values(by='fuzzy_score', ascending=False).to_csv('test_match.csv')

In [237]:
# Define a transformation function for standardization
def standardize_draw_no_bet(value):
    if '1e Helft' in value:
        return 'Draw No Bet - 1e Helft'
    elif '2e Helft' in value:
        return 'Draw No Bet - 2e Helft'
    elif 'Draw No Bet' in value:
        return 'Draw No Bet'
    else:
        return value  # Keep original if no match

# Apply the transformation to the relevant columns
filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)

# Merge the DataFrames using the standardized column
merged_winnaar = pd.merge(
    filtered_toto_winnaar,
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label'], 
    right_on=['event_name', 'standardized_label'], 
    how='inner'  # Perform an inner join
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/3072039294.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/3072039294.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)


In [238]:
# # Filter rows in toto_filtered_football where Market Name == 'Winnaar'
# toto_filtered_football = toto_filtered_football[toto_filtered_football['Market Name'].str.contains('Draw No Bet')]

# Perform a left join on the matched_event_name (from toto) and event_name (from kambi)
merged_df_winnaar = filtered_toto_winnaar.merge(
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label', 'sex', 'start_time'], 
    right_on=['event_name', 'standardized_label', 'sex', 'start_time'], 
    how='left'
)

merged_df_winnaar = merged_df_winnaar[(merged_df_winnaar['matched_event'].notnull())&(merged_df_winnaar['Outcome SubType'] != merged_df_winnaar['outcome_label'])]

In [239]:
merged_df_winnaar

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,match_name,home_team,away_team,start_time,sex,Team1_x,Team2_x,matched_event,fuzzy_score,standardized_label,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y
1,Benfica vs Moreirense,Draw No Bet,Benfica,1.04,1,25,DN,1,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica,Moreirense,Benfica vs Moreirense,100.0,Draw No Bet,2.490446e+09,1.001160e+09,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2.0,Wedstrijd,Match,1.021176e+09,3.630118e+09,2,2,11.00,NaN,Moreirense,2,2025-02-05T04:24:31Z,10/1,1000.0,OPEN,DISABLED,NaN,NaN,Benfica vs Moreirense,FOOTBALL,Primeira Liga,Benfica,Moreirense
2,Benfica vs Moreirense,Draw No Bet,Moreirense,8.00,7,1,DN,2,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica,Moreirense,Benfica vs Moreirense,100.0,Draw No Bet,2.490446e+09,1.001160e+09,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2.0,Wedstrijd,Match,1.021176e+09,3.630118e+09,1,1,1.04,NaN,Benfica,1,2025-02-05T04:24:31Z,1/25,-2500.0,OPEN,DISABLED,NaN,NaN,Benfica vs Moreirense,FOOTBALL,Primeira Liga,Benfica,Moreirense
4,Benfica vs Moreirense,2e Helft: Draw No Bet,Moreirense,6.25,21,4,DN,2,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica,Moreirense,Benfica vs Moreirense,100.0,Draw No Bet - 2e Helft,2.490446e+09,1.001421e+09,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2.0,Wedstrijd,Match,1.021176e+09,3.630118e+09,1,1,1.07,NaN,Benfica,1,2025-02-05T04:24:31Z,1/16,-1430.0,OPEN,DISABLED,NaN,NaN,Benfica vs Moreirense,FOOTBALL,Primeira Liga,Benfica,Moreirense
7,Benfica vs Moreirense,2e Helft: Draw No Bet,Benfica,1.08,2,25,DN,1,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica,Moreirense,Benfica vs Moreirense,100.0,Draw No Bet - 2e Helft,2.490446e+09,1.001421e+09,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2.0,Wedstrijd,Match,1.021176e+09,3.630118e+09,2,2,5.30,NaN,Moreirense,2,2025-02-05T04:24:31Z,17/4,430.0,OPEN,DISABLED,NaN,NaN,Benfica vs Moreirense,FOOTBALL,Primeira Liga,Benfica,Moreirense
8,Tottenham Hotspur vs Manchester United,2e Helft: Draw No Bet,Manchester United,1.98,49,50,DN,2,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester United,Tottenham Hotspur,Manchester United,2025-02-16T16:30:00Z,M,Tottenham Hotspur,Manchester United,Tottenham vs Manchester United,100.0,Draw No Bet - 2e Helft,2.491540e+09,1.001421e+09,Draw No Bet - 2e Helft,Draw No Bet - 2nd Half,GOALS,NaN,2.0,Wedstrijd,Match,1.021043e+09,3.634292e+09,1,1,1.73,NaN,Tottenham,1,2025-02-05T15:48:37Z,8/11,-137.0,OPEN,DISABLED,NaN,NaN,Tottenham vs Manchester United,FOOTBALL,Premier League,Tottenham,Manchester United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5225,Uberlandia EC MG vs EC Democrata MG,1e Helft: Draw No Bet,Uberlandia EC MG,1.59,59,100,DN,1,Voetbal,Brazilië Mineiro Modulo I,Uberlandia EC MG vs EC Democrata MG,Uberlandia EC MG,EC Democrata MG,2025-02-06T23:00:00Z,M,Uberlandia EC MG,EC Democrata MG,Uberlandia MG vs Democrata GV MG,94.5,Draw No Bet - 1e Helft,2.490824e+09,1.001160e+09,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2.0,Wedstrijd,Match,1.022361e+09,3.631396e+09,2,2,2.04,NaN,Democrata GV-MG,2,2025-02-05T17:58:36Z,26/25,104.0,OPEN,ENABLED,NaN,NaN,Uberlandia MG vs Democrata GV MG,FOOTBALL,Mineiro,Uberlandia MG,Democrata GV MG
5226,

## Over/Under

In [240]:
# # filter down to suitable betting opps
# kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

# kambi_filtered_football = kambi_raw_football
# toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football

# Remove duplicates in toto_filtered_basketbal based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_basketbal based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

In [241]:
kambi_filtered_football_overunder = kambi_filtered_football[kambi_filtered_football['bet_offer_type_name'].str.contains('Over')]
toto_filtered_football_overunder = toto_filtered_football[
    (toto_filtered_football['Outcome Name'].str.contains('Over')) |
    (toto_filtered_football['Outcome Name'].str.contains('Under')) |
    (toto_filtered_football['Outcome Name'].str.match(r'^\d+\+$')) |  # matches strings that ONLY contain digit(s) followed by '+'
    (toto_filtered_football['Outcome Name'].str.contains(r'\d{1,2} of meer')) &
    ~(toto_filtered_football['Outcome Name'].str.contains(' en ')) &  # excludes strings containing ' en '
    ~(toto_filtered_football['Outcome Name'].str.contains('&'))  # excludes strings containing '&'
]

In [242]:
set(toto_filtered_football_overunder['Outcome Name'])

{'Atlas en Under',
 'Dundee United en Under',
 'St Mirren wint, beide teams scoren & Over 3.5 doelpunten',
 'Gelijkspel of Hull City en Under',
 'Fortuna Sittard of Ajax en Over',
 'Salford City wint, Over 3.5 doelpunten & Cole Stockton scoort exact 2 doelpunten',
 'Standard Liege wint, beide teams scoren & Over 3.5 doelpunten',
 'Augsburg wint, Over 1.5 doelpunten in de 1e helft & Over 3.5 doelpunten',
 'Clermont Foot en Under',
 'Manchester United of Leicester City en Over',
 'Weymouth FC en Over',
 'KV Kortrijk of Gelijkspel en Over',
 'Hoffenheim wint, beide teams scoren & Over 3.5 doelpunten',
 'Necaxa wint, Over 1.5 doelpunten in de 1e helft & Over 3.5 doelpunten',
 'RSC Anderlecht of Gelijkspel en Under',
 'AC Milan en Under',
 'Monaco wint, Over 1.5 doelpunten in de 1e helft & Over 3.5 doelpunten',
 'Al-Shabab wint, beide teams scoren & Over 3.5 doelpunten',
 'Caernarfon Town FC en Over',
 'Aubagne FC en Over',
 'Panserraikos wint, Over 1.5 doelpunten in de 1e helft & Over 3.5 

In [243]:
toto_filtered_football['Market Name'].unique()

array(['Dubbele Kans', 'Resultaat', 'Draw No Bet', ...,
       'AS Giana Erminio Aantal Goals - Over/Under 1.5',
       'Union Clodiense Chioggia SSD Geen Tegendoelpunt',
       'Union Clodiense Chioggia SSD Aantal Goals - Over/Under 0.5'],
      dtype=object)

In [244]:
import unicodedata

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_english_label'] = kambi_filtered_football_overunder['criterion_english_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Event Name'] = toto_filtered_football_overunder['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Market Name'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

kambi_events = kambi_filtered_football_overunder['event_name'].tolist()

# # Function to find the best match for 'Event Name' in toto_filtered_football
# def find_best_match(event_name):
#     # Use a token-based similarity for better handling of substrings
#     result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=100)
#     if result is None:  # No match found
#         return None
#     match, score = result
#     return match


# # Apply matching function to toto_filtered_football
# toto_filtered_football_overunder['matched_event'] = toto_filtered_football_overunder['Event Name'].apply(find_best_match)
# Perform a left join to bring in 'matched_event' and 'fuzzy_score' from matched_events
toto_filtered_football_overunder = toto_filtered_football_overunder.merge(
    matched_events,
    on='Event Name',  # Join on the 'Event Name' column
    how='left'  # Ensure all rows in toto_filtered_football_overunder are retained
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/2219985270.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/2219985270.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_r

In [245]:
# Get unique records from 'Event Name' and 'matched_event'
unique_records = toto_filtered_football_overunder[['Event Name', 'matched_event']].drop_duplicates()

# Display the result
unique_records[~unique_records['matched_event'].isna()]

,Event Name,matched_event
0,Benfica vs Moreirense,Benfica vs Moreirense
21,Tottenham Hotspur vs Manchester United,Tottenham vs Manchester United
93,Piast Gliwice vs Legia Warschau,Piast Gliwice vs Legia Warszawa
146,Pogon Szczecin vs Gornik Zabrze,Pogon Szczecin vs Gornik Zabrze
224,Rakow Czestochowa vs GKS Katowice,Rakow Czestochowa vs GKS Katowice
...,...,...
51462,Puebla vs Pumas UNAM,Puebla FC (W) vs Pumas UNAM (W)
52952,Sao Jose EC SP U23 vs CA Votuporanguense SP,Sao Jose Esporte Clube vs Votuporanguense SP
52994,Uberlandia EC MG vs EC Democrata MG,Uberlandia MG vs Democrata GV MG
53113,Pouso Alegre MG vs Betim Futebol,Pouso Alegre MG vs Betim MG


In [246]:
# toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder.csv')
# kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder.csv')  

In [247]:
# Create 'OverUnderType'
kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
        lambda x: 'Goals' if ('Doelpunten' in x and 'Resultaat' not in x) 
        else 'Wedstrijd schoten op doel' if ('Totaal Aantal Schoten op Doel' in x and ' & ' not in x)
        else 'Team schoten op doel' if ('Totaal Aantal Schoten op Doel door' in x and ' & ' not in x) 
        else 'Team schoten' if ('Totaal Aantal Schoten door' in x and ' & ' not in x) 
        else 'Totaal aantal schoten op doel' if ('Totaal Aantal Schoten op Doel' in x and ' & ' not in x)
        else 'Totaal aantal schoten' if ('Totaal Aantal Schoten' in x and ' & ' not in x)
        else 'Speler aantal schoten op doel' if ('Schoten van Speler op Doel' in x and ' & ' not in x)
        else 'Speler aantal schoten' if ('Schoten van Speler' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
         else 'Dubbele Kans' if ('Dubbele Kans' in x or ' en ' in x or ' & ' in x)
        else 'other'
    )

def determine_over_under_time(label):
    label_lower = label.lower()

    if '1e helft' in label_lower:
        return '1e Helft'
    elif '2e helft' in label_lower:
        return '2e Helft'

    # Check if the label contains a time range in the format 'XX:XX YY:YY'
    time_match = re.search(r'\b\d{1,2}:\d{2} \d{1,2}:\d{2}\b', label)
    if time_match:
        return time_match.group()  # Return the matched time range
    
    return 'Full Time'

# Apply the function to the column
kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)

kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)

# Create 'Team1' and 'Team2'
kambi_filtered_football_overunder[['Team1', 'Team2']] = kambi_filtered_football_overunder['event_name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
    lambda row: row['Participant'] if row['OverUnderType'] in ['Speler aantal schoten op doel', 'Speler aantal schoten'] else (
        '1' if row['Team1'] in row['criterion_english_label'] else 
        '2' if row['Team2'] in row['criterion_english_label'] else 
        'Total team 1 and team 2'
    ),
    axis=1
)

# # Create 'OverUnderType2'
# kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
#     lambda row: '1' if row['Team1'] in row['criterion_english_label'] else (
#         '2' if row['Team2'] in row['criterion_english_label'] else 'Total team 1 and team 2'
#     ),
#     axis=1
# )

# # Create 'line'
# kambi_filtered_football_overunder['line'] = kambi_filtered_football_overunder['line'] / 1000

# # Creat odds
# kambi_filtered_football_overunder['odds'] = kambi_filtered_football_overunder['odds'] / 1000

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/1308321763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_23948/1308321763.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)
/va

In [248]:
kambi_filtered_football_overunder[(kambi_filtered_football_overunder['OverUnderTime'] != '1e Helft') & (kambi_filtered_football_overunder['OverUnderTime'] != '2e Helft') & (kambi_filtered_football_overunder['OverUnderTime'] != 'Full Time')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,start_time,sex,OverUnderType,OverUnderTime,Team1,Team2,OverUnderType2
6213,2491662111,1001882272,Totaal Aantal Doelpunten 30:00 39:59,Total Goals 30:00 39:59,GOALS,NaN,6,Over/Onder,Over/Under,1022375693,3634617603,Meer dan,Over,3.50,0.5,NaN,OT_OVER,2025-02-05T20:27:03Z,5/2,250.0,OPEN,ENABLED,NaN,NaN,AC Milan vs Roma,FOOTBALL,Coppa Italia,2025-02-05T20:01:00Z,M,Goals,30:00 39:59,AC Milan,Roma,Total team 1 and team 2
6214,2491662111,1001882272,Totaal Aantal Doelpunten 30:00 39:59,Total Goals 30:00 39:59,GOALS,NaN,6,Over/Onder,Over/Under,1022375693,3634617605,Minder dan,Under,1.27,0.5,NaN,OT_UNDER,2025-02-05T20:27:03Z,27/100,-375.0,OPEN,ENABLED,NaN,NaN,AC Milan vs Roma,FOOTBALL,Coppa Italia,2025-02-05T20:01:00Z,M,Goals,30:00 39:59,AC Milan,Roma,Total team 1 and team 2
6215,2491662112,1004581745,Totaal Aantal Doelpunten door Roma 30:00 39:59,Total Goals by Roma 30:00 39:59,NaN,NaN,6,Over/Onder,Over/Under,1022375693,3634617604,Meer dan,Over,6.75,0.5,NaN,OT_OVER,2025-02-05T20:27:00Z,23/4,575.0,OPEN,ENABLED,NaN,NaN,AC Milan vs Roma,FOOTBALL,Coppa Italia,2025-02-05T20:01:00Z,M,Goals,30:00 39:59,AC Milan,Roma,2
6216,2491662112,1004581745,Totaal Aantal Doelpunten door Roma 30:00 39:59,Total Goals by Roma 30:00 39:59,NaN,NaN,6,Over/Onder,Over/Under,1022375693,3634617607,Minder dan,Under,1.10,0.5,NaN,OT_UNDER,2025-02-05T20:27:00Z,1/10,-1000.0,OPEN,ENABLED,NaN,NaN,AC Milan vs Roma,FOOTBALL,Coppa Italia,2025-02-05T20:01:00Z,M,Goals,30:00 39:59,AC Milan,Roma,2
6217,2491662113,1001882272,Totaal Aantal Doelpunten 30:00 59:59,Total Goals 30:00 59:59,GOALS,NaN,6,Over/Onder,Over/Under,1022375693,3634617606,Meer dan,Over,3.85,1.5,NaN,OT_OVER,2025-02-05T20:27:03Z,14/5,285.0,OPEN,ENABLED,NaN,NaN,AC Milan vs Roma,FOOTBALL,Coppa Italia,2025-02-05T20:01:00Z,M,Goals,30:00 59:59,AC Milan,Roma,Total team 1 and team 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133710,2491652125,1001882272,Totaal Aantal Doelpunten 70:00 79:59,Total Goals 70:00 79:59,GOALS,NaN,6,Over/Onder,Over/Under,1022650826,3634598685,Minder dan,Under,1.02,1.5,NaN,OT_UNDER,2025-02-05T20:25:57Z,1/50,-5000.0,OPEN,ENABLED,NaN,NaN,Peterborough United vs Cheltenham,FOOTBALL,EFL Trophy,2025-02-05T19:02:00Z,M,Goals,70:00 79:59,Peterborough United,Cheltenham,Total team 1 and team 2
133713,2491652127,1004581744,Totaal Aantal Doelpunten door Peterborough Uni...,Total Goals by Peterborough United 70:00 79:59,NaN,NaN,6,Over/Onder,Over/Under,1022650826,3634598688,Meer dan,Over,4.40,0.5,NaN,OT_OVER,2025-02-05T20:26:52Z,17/5,340.0,OPEN,ENABLED,NaN,NaN,Peterborough United vs Cheltenham,FOOTBALL,EFL Trophy,2025-02-05T19:02:00Z,M,Goals,70:00 79:59,Peterborough United,Cheltenham,1
133714,2491652127,1004581744,Totaal Aantal Doelpunten door Peterborough Uni...,Total Goals by Peterborough United 70:00 79:59,NaN,NaN,6,Over/Onder,Over/Under,1022650826,3634598690,Minder dan,Under,1.18,0.5,NaN,OT_UNDER,2025-02-05T20:26:52Z,1/6,-560.0,OPEN,ENABLED,NaN,NaN,Peterborough United vs Cheltenham,FOOTBALL,EFL Trophy,2025-02-05T19:02:00Z,M,Goals,70:00 79:59,Peterborough United,Cheltenham,1
133715,2491652128,1004581745,Totaal Aantal Doelpunten door Cheltenham 70:...,Total Goals by Cheltenham 70:00 79:59,NaN,NaN,6,Over/Onder,Over/Under,1022650826,3634598689,Meer dan,Over,6.25,0.5,NaN,OT_OVER,2025-02-05T20:26:52Z,21/4,525.0,OPEN,ENABLED,NaN,NaN,Peterborough United vs Cheltenham,FOOTBALL,EFL Trophy,2025-02-05T19:02:00Z,M,Goals,70:00 79:59,Peterborough United,Cheltenham,2


In [249]:
# Create 'OverUnderType'
toto_filtered_football_overunder['OverUnderType'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: 'Goals' if ('Goals' in x and 'Resultaat' not in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
    # else 'Speler schoten op doel' if ('Speler schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
    else 'Wedstrijd schoten op doel' if ('Wedstrijd schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x)
    else 'Team schoten op doel' if ('Team schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x)
    # else 'Schoten op doel van buiten 16 mtr' if ('schoten op doel van buiten 16 mtr' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) # via specials voor later
    else 'Speler aantal schoten op doel' if ('aantal schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) or ('Speler schoten op doel' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x)
    else 'Speler aantal schoten' if ('aantal schoten' in x and 'Dubbele Kans' not in x and ' en ' not in x and ' & ' not in x) 
    else 'Dubbele Kans' if ('Dubbele Kans' in x or ' en ' in x or ' & ' in x)
    else 'other'
)

# Create 'OverUnderTime'
toto_filtered_football_overunder['OverUnderTime'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: '1e Helft' if '1e helft' in x.lower() else '2e Helft' if '2e helft' in x.lower() 
    else  '00:00 09:59' if 'eerste 10 minuten' in x.lower() else 'Full Time'
)

# Create 'Team1' and 'Team2'
toto_filtered_football_overunder[['Team1', 'Team2']] = toto_filtered_football_overunder['Event Name'].str.split(' vs ', expand=True)

# # Create 'OverUnderType2'
# toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
#     lambda row: '1' if row['Team1'] in row['Market Name'] else (
#         '2' if row['Team2'] in row['Market Name'] else 'Total team 1 and team 2'
#     ),
#     axis=1
# )

# Create 'OverUnderType2'
toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
    lambda row: (
        '1' if row['OverUnderType'] == 'Goals' and row['Team1'] in row['Market Name'] else
        '2' if row['OverUnderType'] == 'Goals' and row['Team2'] in row['Market Name'] else
        'Total team 1 and team 2' if row['OverUnderType'] == 'Goals' else
        'Total team 1 and team 2' if row['OverUnderType'] == 'Wedstrijd schoten op doel' else
        '1' if row['OverUnderType'] == 'Team schoten op doel' and row['Team1'] in row['Outcome Name'] else
        '2' if row['OverUnderType'] == 'Team schoten op doel' and row['Team2'] in row['Outcome Name'] else
        row['Market Name'].split('aantal schoten')[0].strip() if row['OverUnderType'] in ['Speler aantal schoten op doel', 'Speler aantal schoten'] else # Get player name
        row['OverUnderType']  # Default to OverUnderType if none of the conditions apply
    ),
    axis=1
)

# # Create 'line'
# toto_filtered_football_overunder['line'] = toto_filtered_football_overunder['Market Name'].apply(
#     lambda x: float(x.split('Over/Under ')[-1]) if 'Over/Under ' in x else None
# )

def extract_line_toto(row):
    market_name = row['Market Name']
    outcome_name = row['Outcome Name']
    over_under_type = row['OverUnderType']

    # Extract number from 'Market Name' when it contains 'Over/Under '
    if 'Over/Under ' in market_name:
        return float(market_name.split('Over/Under ')[-1])

    # Extract number from 'Outcome Name'
    match = re.search(r'\d+(\.\d+)?', outcome_name)
    if match:
        value = float(match.group())  # Convert extracted number to float

        # Adjust value based on conditions
        if over_under_type in ['Wedstrijd schoten op doel', 'Team schoten op doel'] and 'of meer' in outcome_name:
            return value - 0.5
        elif over_under_type in ['Speler aantal schoten op doel', 'Speler aantal schoten'] and '+' in outcome_name:
            return value - 0.5
        else:
            return value

    return None  # Default if no number found

# Apply function to create 'line' column
toto_filtered_football_overunder['line'] = toto_filtered_football_overunder.apply(extract_line_toto, axis=1)

In [250]:
toto_filtered_football_overunder[toto_filtered_football_overunder['OverUnderType'] == 'Goals']

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition,match_name,home_team,away_team,start_time,sex,matched_event,fuzzy_score,OverUnderType,OverUnderTime,Team1,Team2,OverUnderType2,line
0,Benfica vs Moreirense,Aantal Goals Over/Under 2.5,Under,2.30,13,10,HL,L,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica vs Moreirense,100.0,Goals,Full Time,Benfica,Moreirense,Total team 1 and team 2,2.5
1,Benfica vs Moreirense,Aantal Goals Over/Under 2.5,Over,1.54,27,50,HL,1,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica vs Moreirense,100.0,Goals,Full Time,Benfica,Moreirense,Total team 1 and team 2,2.5
2,Benfica vs Moreirense,Aantal Goals Over/Under 3.5,Under,1.51,51,100,HL,L,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica vs Moreirense,100.0,Goals,Full Time,Benfica,Moreirense,Total team 1 and team 2,3.5
3,Benfica vs Moreirense,Aantal Goals Over/Under 3.5,Over,2.35,27,20,HL,1,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica vs Moreirense,100.0,Goals,Full Time,Benfica,Moreirense,Total team 1 and team 2,3.5
4,Benfica vs Moreirense,Aantal Goals Over/Under 1.5,Under,4.50,7,2,HL,L,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica vs Moreirense,100.0,Goals,Full Time,Benfica,Moreirense,Total team 1 and team 2,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56530,Union Clodiense Chioggia SSD vs AS Giana Erminio,AS Giana Erminio Aantal Goals Over/Under 1.5,Under,1.44,11,25,HL,L,Voetbal,Italy Serie C,Union Clodiense Chioggia SSD vs AS Giana Erminio,Union Clodiense Chioggia SSD,AS Giana Erminio,2025-02-08T14:00:00Z,M,None,NaN,Goals,Full Time,Union Clodiense Chioggia SSD,AS Giana Erminio,2,1.5
56531,Union Clodiense Chioggia SSD vs AS Giana Erminio,Aantal Goals Over/Under 2.5,Over,1.91,91,100,HL,1,Voetbal,Italy Serie C,Union Clodiense Chioggia SSD vs AS Giana Erminio,Union Clodiense Chioggia SSD,AS Giana Erminio,2025-02-08T14:00:00Z,M,None,NaN,Goals,Full Time,Union Clodiense Chioggia SSD,AS Giana Erminio,Total team 1 and team 2,2.5
56532,Union Clodiense Chioggia SSD vs AS Giana Erminio,Aantal Goals Over/Under 2.5,Under,1.73,73,100,HL,L,Voetbal,Italy Serie C,Union Clodiense Chioggia SSD vs AS Giana Erminio,Union Clodiense Chioggia SSD,AS Giana Erminio,2025-02-08T14:00:00Z,M,None,NaN,Goals,Full Time,Union Clodiense Chioggia SSD,AS Giana Erminio,Total team 1 and team 2,2.5
56533,Union Clodiense Chioggia SSD vs AS Giana Erminio,Union Clodiense Chioggia SSD Aantal Goals Ov...,Over,1.29,29,100,HL,1,Voetbal,Italy Serie C,Union Clodiense Chioggia SSD vs AS Giana Erminio,Union Clodiense Chioggia SSD,AS Giana Erminio,2025-02-08T14:00:00Z,M,None,NaN,Goals,Full Time,Union Clodiense Chioggia SSD,AS Giana Erminio,1,0.5


In [261]:
# Perform the merge
merged_football_overunder = pd.merge(
    toto_filtered_football_overunder,
    kambi_filtered_football_overunder,
    left_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'matched_event', 'sex', 'start_time'],
    right_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'event_name', 'sex', 'start_time'],
    how='inner'  # Use 'inner' join to get only matching rows; adjust as needed
)

merged_football_overunder['Outcome Name cleaned'] = merged_football_overunder['Outcome Name'].apply(
    lambda x: 'Over' if re.match(r'^\d{1,2} of meer', x) else x
)

# Keep records with opposite outcomes
merged_football_overunder = merged_football_overunder[merged_football_overunder['outcome_english_label'] != merged_football_overunder['Outcome Name cleaned']] # Dubbel check voor Schoten OverUnderType & voer fuzzymatching uit op speler.

# Display the merged DataFrame
merged_football_overunder.head()

,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,match_name,home_team,away_team,start_time,sex,matched_event,fuzzy_score,OverUnderType,OverUnderTime,Team1_x,Team2_x,OverUnderType2,line,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y,Outcome Name cleaned
0,Benfica vs Moreirense,Moreirense Aantal Goals Over/Under 1.5,Under,1.08,2,25,HL,L,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica vs Moreirense,100.0,Goals,Full Time,Benfica,Moreirense,2,1.5,2490446039,1001159633,Totaal Aantal Doelpunten door Moreirense,Total Goals by Moreirense,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021176419,3630117367,Meer dan,Over,5.80,NaN,OT_OVER,2025-02-05T04:24:30Z,24/5,480.0,OPEN,ENABLED,NaN,NaN,Benfica vs Moreirense,FOOTBALL,Primeira Liga,Benfica,Moreirense,Under
3,Benfica vs Moreirense,Moreirense Aantal Goals Over/Under 1.5,Over,6.50,11,2,HL,1,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica vs Moreirense,100.0,Goals,Full Time,Benfica,Moreirense,2,1.5,2490446039,1001159633,Totaal Aantal Doelpunten door Moreirense,Total Goals by Moreirense,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021176419,3630117368,Minder dan,Under,1.09,NaN,OT_UNDER,2025-02-05T03:12:35Z,1/12,-1115.0,OPEN,ENABLED,NaN,NaN,Benfica vs Moreirense,FOOTBALL,Primeira Liga,Benfica,Moreirense,Over
5,Benfica vs Moreirense,Benfica Aantal Goals Over/Under 3.5,Over,3.40,12,5,HL,1,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica vs Moreirense,100.0,Goals,Full Time,Benfica,Moreirense,1,3.5,2490446047,1001159967,Totaal Aantal Doelpunten door Benfica,Total Goals by Benfica,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021176419,3630117385,Minder dan,Under,1.26,NaN,OT_UNDER,2025-02-05T03:12:35Z,1/4,-385.0,OPEN,ENABLED,NaN,NaN,Benfica vs Moreirense,FOOTBALL,Primeira Liga,Benfica,Moreirense,Over
6,Benfica vs Moreirense,Benfica Aantal Goals Over/Under 3.5,Under,1.29,29,100,HL,L,Voetbal,Portugal Primeira Liga,Benfica vs Moreirense,Benfica,Moreirense,2025-02-08T18:00:00Z,M,Benfica vs Moreirense,100.0,Goals,Full Time,Benfica,Moreirense,1,3.5,2490446047,1001159967,Totaal Aantal Doelpunten door Benfica,Total Goals by Benfica,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021176419,3630117381,Meer dan,Over,3.40,NaN,OT_OVER,2025-02-05T03:12:35Z,12/5,240.0,OPEN,ENABLED,NaN,NaN,Benfica vs Moreirense,FOOTBALL,Primeira Liga,Benfica,Moreirense,Under
9,Tottenham Hotspur vs Manchester United,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,Over,1.71,71,100,HL,1,Voetbal,Engeland Premier League,Tottenham Hotspur vs Manchester United,Tottenham Hotspur,Manchester United,2025-02-16T16:30:00Z,M,Tottenham vs Manchester United,100.0,Goals,1e Helft,Tottenham Hotspur,Manchester United,1,0.5,2489085191,1003194958,Totaal Aantal Doelpunten door Tottenham 1e H...,Total Goals by Tottenham 1st Half,NaN,NaN,6,Over/Onder,Over/Under,1021042755,3625061270,Minder dan,Under,2.00,NaN,OT_UNDER,2025-02-05T15:48:03Z,Evens,100.0,OPEN,ENABLED,NaN,NaN,Tottenham vs Manchester United,FOOTBALL,Premier League,Tottenham,Manchester United,Over


## Match Winnaar & OverUnder and do Arb Analysis

In [262]:
# Perform the stacked union
total_football = pd.concat([merged_football_overunder, merged_df_winnaar], ignore_index=True, sort=True)
total_football

,Event Name,Market Name,Odds (Decimal),Outcome Name,Outcome Name cleaned,Outcome SubType,Outcome Type,OverUnderTime,OverUnderType,OverUnderType2,Price Denominator,Price Numerator,Team1_x,Team1_y,Team2_x,Team2_y,away_score,away_team,bet_offer_id,bet_offer_type_english_name,bet_offer_type_id,bet_offer_type_name,cash_out_status,changed_date,competition,criterion_english_label,criterion_id,criterion_label,event_id,event_name,fuzzy_score,group_name,home_score,home_team,lifetime,line,match_name,matched_event,occurrence_type,odds,odds_american,odds_fractional,outcome_english_label,outcome_id,outcome_label,participant,sex,sport_x,sport_y,standardized_label,start_time,status,type
0,Benfica vs Moreirense,Moreirense Aantal Goals Over/Under 1.5,1.08,Under,Under,L,HL,Full Time,Goals,2,25,2,Benfica,Benfica,Moreirense,Moreirense,NaN,Moreirense,2.490446e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-05T04:24:30Z,Portugal Primeira Liga,Total Goals by Moreirense,1.001160e+09,Totaal Aantal Doelpunten door Moreirense,1.021176e+09,Benfica vs Moreirense,100.0,Primeira Liga,NaN,Benfica,FULL_TIME,1.5,Benfica vs Moreirense,Benfica vs Moreirense,NaN,5.80,480.0,24/5,Over,3.630117e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-08T18:00:00Z,OPEN,OT_OVER
1,Benfica vs Moreirense,Moreirense Aantal Goals Over/Under 1.5,6.50,Over,Over,1,HL,Full Time,Goals,2,2,11,Benfica,Benfica,Moreirense,Moreirense,NaN,Moreirense,2.490446e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-05T03:12:35Z,Portugal Primeira Liga,Total Goals by Moreirense,1.001160e+09,Totaal Aantal Doelpunten door Moreirense,1.021176e+09,Benfica vs Moreirense,100.0,Primeira Liga,NaN,Benfica,FULL_TIME,1.5,Benfica vs Moreirense,Benfica vs Moreirense,NaN,1.09,-1115.0,1/12,Under,3.630117e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-08T18:00:00Z,OPEN,OT_UNDER
2,Benfica vs Moreirense,Benfica Aantal Goals Over/Under 3.5,3.40,Over,Over,1,HL,Full Time,Goals,1,5,12,Benfica,Benfica,Moreirense,Moreirense,NaN,Moreirense,2.490446e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-05T03:12:35Z,Portugal Primeira Liga,Total Goals by Benfica,1.001160e+09,Totaal Aantal Doelpunten door Benfica,1.021176e+09,Benfica vs Moreirense,100.0,Primeira Liga,NaN,Benfica,FULL_TIME,3.5,Benfica vs Moreirense,Benfica vs Moreirense,NaN,1.26,-385.0,1/4,Under,3.630117e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-08T18:00:00Z,OPEN,OT_UNDER
3,Benfica vs Moreirense,Benfica Aantal Goals Over/Under 3.5,1.29,Under,Under,L,HL,Full Time,Goals,1,100,29,Benfica,Benfica,Moreirense,Moreirense,NaN,Moreirense,2.490446e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-05T03:12:35Z,Portugal Primeira Liga,Total Goals by Benfica,1.001160e+09,Totaal Aantal Doelpunten door Benfica,1.021176e+09,Benfica vs Moreirense,100.0,Primeira Liga,NaN,Benfica,FULL_TIME,3.5,Benfica vs Moreirense,Benfica vs Moreirense,NaN,3.40,240.0,12/5,Over,3.630117e+09,Meer dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-08T18:00:00Z,OPEN,OT_OVER
4,Tottenham Hotspur vs Manchester United,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,1.71,Over,Over,1,HL,1e Helft,Goals,1,100,71,Tottenham Hotspur,Tottenham,Manchester United,Manchester United,NaN,Manchester United,2.489085e+09,Over/Under,6.0,Over/Onder,ENABLED,2025-02-05T15:48:03Z,Engeland Premier League,Total Goals by Tottenham 1st Half,1.003195e+09,Totaal Aantal Doelpunten door Tottenham 1e H...,1.021043e+09,Tottenham vs Manchester United,100.0,Premier League,NaN,Tottenham Hotspur,NaN,0.5,Tottenham Hotspur vs Manchester United,Tottenham vs Manchester United,NaN,2.00,100.0,Evens,Under,3.625061e+09,Minder dan,NaN,M,Voetbal,FOOTBALL,NaN,2025-02-16T16:30:00Z,OPEN,OT_UNDER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,Uberlandia EC MG vs EC Democrata MG,1e Helft: Draw No Bet,1.59,Uberlandia EC MG,NaN,1,DN,NaN,NaN,NaN,100,59,Uberlandia EC MG,Uberlandia MG,EC Democrata MG,Democrata 

In [263]:
# Calculate Arbitrage Percentage
total_football['Arbitrage Percentage'] = (1 / total_football['Odds (Decimal)'] + 1 / total_football['odds']) * 100

# Identify Arbitrage Opportunities
total_football['Is Arbitrage'] = total_football['Arbitrage Percentage'] < 100

# Calculate Optimal Stakes if Arbitrage
total_stake = 1000
total_football['Stake A'] = total_football.apply(
    lambda row: (1 / row['Odds (Decimal)'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)
total_football['Stake B'] = total_football.apply(
    lambda row: (1 / row['odds'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)

total_football[['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]    

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
0,Benfica vs Moreirense,Moreirense Aantal Goals Over/Under 1.5,Under,Meer dan,1.08,5.80,109.833972,False,0,0
1,Benfica vs Moreirense,Moreirense Aantal Goals Over/Under 1.5,Over,Minder dan,6.50,1.09,107.127735,False,0,0
2,Benfica vs Moreirense,Benfica Aantal Goals Over/Under 3.5,Over,Minder dan,3.40,1.26,108.776844,False,0,0
3,Benfica vs Moreirense,Benfica Aantal Goals Over/Under 3.5,Under,Meer dan,1.29,3.40,106.931145,False,0,0
4,Tottenham Hotspur vs Manchester United,Tottenham Hotspur 1e Helft: Aantal Goals Ove...,Over,Minder dan,1.71,2.00,108.479532,False,0,0
...,...,...,...,...,...,...,...,...,...,...
4882,Uberlandia EC MG vs EC Democrata MG,1e Helft: Draw No Bet,Uberlandia EC MG,2,1.59,2.04,111.912690,False,0,0
4883,Uberlandia EC MG vs EC Democrata MG,2e Helft: Draw No Bet,EC Democrata MG,NaN,2.10,NaN,NaN,False,0,0
4884,Uberlandia EC MG vs EC Democrata MG,2e Helft: Draw No Bet,Uberlandia EC MG,NaN,1.61,NaN,NaN,False,0,0
4885,Villa Nova AC MG vs Itabirito FC MG,Draw No Bet,Itabirito FC MG,1,1.98,1.77,107.002226,False,0,0


In [264]:
total_football[total_football['Is Arbitrage'] == True][['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]  

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B


In [268]:
total_football[(total_football['Is Arbitrage'] == True)][['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'line', 'outcome_english_label', 'type', 'Outcome SubType', 'OverUnderType2',
       'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       'OverUnderType', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       'Is Arbitrage', 'Stake A', 'Stake B', 'group_name', 'sex', 'start_time']]

,Event Name,matched_event,Market Name,criterion_label,Outcome Name,line,outcome_english_label,type,Outcome SubType,OverUnderType2,Odds (Decimal),odds,Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,OverUnderType,OverUnderTime,Team1_x,Team2_x,Team1_y,Team2_y,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B,group_name,sex,start_time


In [269]:
total_football[total_football['OverUnderType'] == 'Wedstrijd schoten op doel'][['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'Outcome Name cleaned', 'line', 'outcome_english_label', 'type', 'Outcome SubType', 'OverUnderType2',
       'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       'OverUnderType', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       'Is Arbitrage', 'Stake A', 'Stake B', 'group_name', 'sex', 'start_time']]

,Event Name,matched_event,Market Name,criterion_label,Outcome Name,Outcome Name cleaned,line,outcome_english_label,type,Outcome SubType,OverUnderType2,Odds (Decimal),odds,Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,OverUnderType,OverUnderTime,Team1_x,Team2_x,Team1_y,Team2_y,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B,group_name,sex,start_time
2958,Liverpool vs Tottenham Hotspur,Liverpool vs Tottenham,Wedstrijd schoten op doel,Totaal Aantal Schoten op Doel (Afgehandeld Vol...,10 of meer schoten op doel,Over,9.5,Under,OT_UNDER,NaN,Total team 1 and team 2,1.25,3.65,1,4,--,NaN,Voetbal,England League Cup,Wedstrijd schoten op doel,Full Time,Liverpool,Tottenham Hotspur,Liverpool,Tottenham,107.39726,False,0,0,EFL Cup,M,2025-02-06T20:00:00Z
